```
The adv instruction (opcode 0) performs division. The numerator is the value in the A register. The denominator is found by raising 2 to the power of the instruction's combo operand. (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) The result of the division operation is truncated to an integer and then written to the A register.

The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand, then stores the result in register B.

The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to the B register.

The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register is not zero, it jumps by setting the instruction pointer to the value of its literal operand; if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.

The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C, then stores the result in register B. (For legacy reasons, this instruction reads an operand but ignores it.)

The out instruction (opcode 5) calculates the value of its combo operand modulo 8, then outputs that value. (If a program outputs multiple values, they are separated by commas.)

The bdv instruction (opcode 6) works exactly like the adv instruction except that the result is stored in the B register. (The numerator is still read from the A register.)

The cdv instruction (opcode 7) works exactly like the adv instruction except that the result is stored in the C register. (The numerator is still read from the A register.)
```


    Combo operands 0 through 3 represent literal values 0 through 3. 
    Combo operand 4 represents the value of register A.
    Combo operand 5 represents the value of register B.
    Combo operand 6 represents the value of register C.
    Combo operand 7 is reserved and will not appear in valid programs.


Register A: 729 \
Register B: 0 \
Register C: 0 \
Program: 0,1,5,4,3,0

In [44]:
opcodes = {0 : "adv", 
           1 : "bxl", 
           2 : "bst", 
           3 : "jnz", 
           4 : "bxc", 
           5 : "out", 
           6 : "bdv",
           7 : "cdv"}

In [45]:
def get_combo_operand(operand, state):
    match operand:
        case 0: combo_operand = 0
        case 1: combo_operand = 1
        case 2: combo_operand = 2
        case 3: combo_operand = 3
        case 4: combo_operand = state["A"]
        case 5: combo_operand = state["B"]
        case 6: combo_operand = state["C"]
        case _: print("Invalid operand")
    return combo_operand

def print_state(pc, operand, combo_operand, state):
    print(f"PC: {pc}, Operand: {operand}, Combo Operand: {combo_operand}, A: {state['A']}, B: {state['B']}, C: {state['C']}")
    print(f"\t A: {state['A']}, B: {state['B']}, C: {state['C']}")
    
    
def disassemble(program, opcodes):
    pc = 0
    while pc < len(program):
        opcode = program[pc]
        operand = program[pc + 1]
        if operand <=3: combo_operand = operand
        elif operand == 4: combo_operand = "A"
        elif operand == 5: combo_operand = "B"
        elif operand == 6: combo_operand = "C"
        if opcode in [0,2,5,6,7]: # combo instruction
            print(f"{opcodes[opcode]} {combo_operand}")
        elif opcode in [1,3]: # literal operand
            print(f"{opcodes[opcode]} {operand}")
        elif opcode == 4: # no operand
            print(f"{opcodes[opcode]}")
        pc += 2
      

In [46]:
def run(program, state):  
    output = []  
    pc = 0
    printed_flag = False
    jmp_flag = False
    while True:
        opcode, operand = program[pc], program[pc+1]
        combo_operand = get_combo_operand(operand, state)
        # print_state(pc, operand, combo_operand, state)
        if opcode == 0:
            state["A"] = int(state["A"] / 2**combo_operand)
        elif opcode == 1:
            state["B"] = state["B"] ^ operand
        elif opcode == 2:
            state["B"] = combo_operand % 8
        elif opcode == 3:
            if state["A"] != 0: 
                pc = operand
                jmp_flag = True
        elif opcode == 4:
            state["B"] = state["B"] ^ state["C"]
        elif opcode == 5:
            # if printed_flag: print(",", end="")
            # print(combo_operand % 8, end="")
            output.append(combo_operand % 8)
            printed_flag = True
        elif opcode == 6:
            state["B"] = int(state["A"] / 2**combo_operand)
        elif opcode == 7:
            state["C"] = int(state["A"] / 2**combo_operand)
        else: print("Unknown opcode")
        
        if not jmp_flag: 
            pc += 2
        else:
            jmp_flag = False
        if pc >= len(program): break
    return state, output


In [47]:
# state = {"A" : 729, "B" : 0, "C" : 0} # part 1 test
# program = [0,1,5,4,3,0]
# state = {"A" : 117440, "B" : 0, "C" : 0} # part 2 test
# program = [0,3,5,4,3,0]
state = {"A" : 23999685, "B" : 0, "C" : 0}
program = [2,4,1,1,7,5,1,5,0,3,4,4,5,5,3,0]
state, output = run(program, state)

print(output)
print("--------------")    
print(f"Final state: {state}")
print("--------------")
disassemble(program, opcodes)

[5, 0, 3, 5, 7, 6, 1, 5, 4]
--------------
Final state: {'A': 0, 'B': 4, 'C': 1}
--------------
bst A
bxl 1
cdv B
bxl 5
adv 3
bxc
out B
jnz 0


In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["Output"])
for i in range(0, 10000):
    state = {"A" : i, "B" : 0, "C" : 0}
    state, output = run(program, state)
    # print(f"i: {i}, output: {output}")
    if output[0] == program[0]:
        df.loc[i] = {"Output": output}
df

,Output
0,[4]
1,[4]
2,[6]
3,[7]
4,[0]
...,...
9995,"[5, 4, 4, 3, 6]"
9996,"[0, 4, 4, 3, 6]"
9997,"[1, 4, 4, 3, 6]"
9998,"[4, 4, 4, 3, 6]"
